In [1]:
f = open(r"./data/monash/monash-df.pkl", "rb")

In [2]:
import pickle


def yield_data(pickle_file_path="./data/monash/monash-df.pkl"):
    """
    Generator function to yield objects one at a time from a pickle file.

    Args:
        pickle_file_path (str): Path to the pickle file.

    Yields:
        dict: A dictionary with file name, df, freq as keys.

    Raises:
        FileNotFoundError: If the pickle file doesn't exist.
        pickle.UnpicklingError: If the pickle file is corrupted.
    """
    try:
        with open(pickle_file_path, "rb") as f:
            while True:
                try:
                    obj = pickle.load(f)
                    items = [i for i in obj.items()][0]
                    yield {
                        "name": items[0],
                        "df": items[1][0],
                        "freq": items[1][1],
                    }
                except EOFError:
                    break
    except FileNotFoundError:
        raise FileNotFoundError(f"Pickle file not found: {pickle_file_path}")
    except pickle.UnpicklingError:
        raise pickle.UnpicklingError("Corrupted pickle file.")

In [3]:
data_generator = yield_data()

In [4]:
import pandas as pd
import numpy as np


def prepare_time_series(df, frequency):
    """
    Convert DataFrame with series_value lists to a time series DataFrame, handling varied frequencies.

    Args:
        df (pd.DataFrame): DataFrame with series_name, start_timestamp, series_value.
        frequency (str): Frequency of the series (e.g., '4_seconds', 'half_hourly', 'daily').

    Returns:
        pd.DataFrame: DataFrame with timestamps as index and series_name as columns.

    Raises:
        ValueError: If the frequency is unsupported.
    """
    freq_map = {
        '4_seconds': '4s',
        'minutely': 'min',
        'hourly': 'h',
        'half_hourly': '30min',
        'daily': 'D',
        'weekly': 'W',
        'monthly': 'ME',
        'quarterly': 'Q',
        'yearly': 'Y'
    }

    pandas_freq = freq_map.get(frequency)
    series_dict = {}
    for _, row in df.iterrows():
        series_name = row['series_name']
        start_time = pd.to_datetime(row['start_timestamp'])
        values = row['series_value']
        timestamps = pd.date_range(
            start=start_time, periods=len(values), freq=pandas_freq)
        series_dict[series_name] = pd.Series(values, index=timestamps)
    ts_df = pd.DataFrame(series_dict)
    return ts_df

In [19]:
missing_indices = []

for idx, item in enumerate(data_generator):
    df = item['df']
    name = item['name']

    # Print the column names
    print(f"DataFrame {idx} ('{name}') columns:", df.columns.tolist())

    # Check for missing 'start_timestamp'
    if 'start_timestamp' not in df.columns:
        missing_indices.append(idx)

print("\nIndices of DataFrames missing 'start_timestamp':", missing_indices)


DataFrame 0 ('m3_monthly_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 1 ('covid_deaths_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 2 ('us_births_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 3 ('weather_dataset.tsf') columns: ['series_name', 'series_type', 'series_value']
DataFrame 4 ('m4_monthly_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 5 ('m1_quarterly_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 6 ('kaggle_web_traffic_weekly_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 7 ('dominick_dataset.tsf') columns: ['series_name', 'series_value']
DataFrame 8 ('car_parts_dataset_without_missing_values.tsf') columns: ['series_name', 'start_timestamp', 'series_value']
DataFrame 9 ('m4_weekly_dataset.tsf') columns: ['series_name', 'start_timestamp', 'series_value']

In [7]:
i = 0
for data in data_generator:
    prepare_time_series(data['df'], data['freq'])
    print(i)
    i += 1

0


/tmp/ipykernel_55664/951930512.py:36: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  timestamps = pd.date_range(start=start_time, periods=len(values), freq=pandas_freq)


1
2
3
4
5
6
7
8


KeyError: 'start_timestamp'